In [15]:
#change ovito export xyz file to RBS simulation input data file
#ovito export xyz file format is :
#25200000
#Lattice="0.0 0.0 5824.86343385 0.0 211.4 0.0 215.0 0.0 0.0" Origin="-86.0 -90.6 -2900.7819900573" Properties=pos:R:3:species:S:1:id:I:1
#-79.5354212843 -89.9262116686 -2899.4576111988 Ga 1
#-80.8037735621 -88.2192198017 -2890.5819782791 Ga 2
#-85.4477714422 -86.5041908662 -2899.4191513497 Ga 3
#RBS data file example:
#25200000
#Lattice="215.00000 0.00000 0.00000 0.00000 214.4 0.00000 0.00000 0.00000 5824.86343385"
#Ga -79.5354212843 -89.9262116686 -2899.4576111988  1
#Ga -80.8037735621 -88.2192198017 -2890.5819782791  1
#Ga -80.8037735621 -88.2192198017 -2890.5819782791  1


In [1]:
import pandas as pd
import numpy as np
import os
import re

In [19]:

# Replace 'your_file.txt' with the actual path to your file
file_path = '/home/heruhe/Desktop/Ga2o3/RBS/gamma110/gamma110.in'
file_path='/home/heruhe/Desktop/Ga2o3/combine_cells/cascade_npt/1000PKA'
file_path='/home/heruhe/Desktop/Ga2o3/RBS/Base/600PKAgamma-Ga2O3'
file_path='/Users/ruhe/Desktop/ga2o3/overlapping/overlapping_npt/datafile/cell.800'
try:
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            if i < 3:
                print("Line {}: {}".format(i + 1, line.strip())) # strip() removes leading/trailing whitespace
                
            else:
                break
except FileNotFoundError:
    print("File not found:", file_path)
except Exception as e:
    print("An error occurred:", str(e))

Line 1: 81920
Line 2: Lattice="100.7681457795 0.0 0.0 0.0 0.0 95.1254774817 0.0 95.0228656657 0.0" Properties=pos:R:3:species:S:1
Line 3: 93.0578892825 39.6723080685 28.3994432177 2


In [3]:
df=pd.read_csv(file_path,skiprows=2,header=None,sep='\s+')
df.columns=['x','y','z','type']
df

,x,y,z,type
0,1.790802,1.104321,1.381482,2
1,4.078201,2.873236,98.815332,2
2,0.929698,4.099700,98.958930,1
3,4.926753,5.726208,1.366354,1
4,7.859546,5.630084,98.868102,2
...,...,...,...,...
81915,90.979613,94.037781,95.747593,4
81916,93.113955,92.593627,97.213386,5
81917,94.965818,91.402946,95.780862,5
81918,95.065688,94.017726,95.749560,3


In [20]:

# Function to convert OVITO XYZ format to RBS format for a single line
#Move the cell origin to zero
def convert_xyz_to_rbs(line,bx,by,bz):
    parts = line.split()
    if len(parts) == 4:
        x, y, z,id_val = parts
        x=float(x)-bx
        y=float(y)-by
        z=float(z)-bz
        s=''
        id_val=int(id_val)
        if id_val==1:
            s='GaI'
        if id_val==2:
            s='GaII'
        if id_val==3:
            s='OI'
        if id_val==4:
            s='OII'
        if id_val==5:
            s='OIII'
        
        return f"{s} {x} {y} {z} {id_val}\n"
        
    else:
        return None
    
    



# Replace 'input.xyz' with your input XYZ file and 'output.rbs' with your desired output RBS file
input_file = file_path
output_file = file_path='/Users/ruhe/Desktop/ga2o3/overlapping/overlapping_npt/datafile/800PKA.in'

try:
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        # Copy the first two lines as is (header lines)
        for i, line in enumerate(infile):
            # Copy the first two lines as is (header lines)
            if i ==0:
                #print(line)
                outfile.write(line)
            if i==1:
                # Define a regular expression pattern to match 'Lattice="..."' and capture the content inside the quotes
                pattern = r'Lattice="([^"]*)"'
                # Define a regular expression pattern to match 'Origin="..."' and capture the content inside the quotes
                pattern2 = r'Origin="([^"]*)"'

                # Use re.search() to find the pattern in the input string
                match = re.search(pattern, line)
                match2 = re.search(pattern2, line)
                print(match2)
                # Check if a match was found
                if match or match2:
                    lattice_value =match.group(1) 
                    numbers = [float(x) for x in lattice_value.split()]
                    if match2!=None:
                        origin_value =match2.group(1) 
                        numbers2 = [float(x) for x in origin_value.split()]
                        print(numbers2)
                    else:
                        numbers2=[0,0,0]
                    # Create a 3x3 matrix from the list of numbers
                    matrix = np.array(numbers).reshape(3, 3)
                    
                    # Print the resulting matrix
                    print(matrix)
                    y=matrix[2][1]
                    z=matrix[1][2]
                    x=matrix[0][0]
                    move_z=numbers2[2]
                    move_y=numbers2[1]
                    move_x=numbers2[0]
                    
                    Lattice='Lattice="'+str(x)+' 0.00000'+' 0.00000'+' 0.00000 '+str(y)+' 0.00000'+' 0.00000'+' 0.00000 '+str(z)+'"\n'
                    print(Lattice)
                    outfile.write(Lattice)
                else:
                    print("Lattice not found in the input string.")
            if i>1:
                convert_line=convert_xyz_to_rbs(line,move_x,move_y,move_z)
                outfile.write(convert_line)
                
except FileNotFoundError:
    print("File not found:", input_file)
except Exception as e:
    print("An error occurred:", str(e))

print("Conversion complete.")

None
[[100.76814578   0.           0.        ]
 [  0.           0.          95.12547748]
 [  0.          95.02286567   0.        ]]
Lattice="100.7681457795 0.00000 0.00000 0.00000 95.0228656657 0.00000 0.00000 0.00000 95.1254774817"

Conversion complete.


#change lammps data file to RBS simulation input data file

LAMMPS data file via write_data, version 27 Jun 2024, timestep = 6010, units = metal

3850240 atoms
2 atom types

-0.6081316254454121 97.46470870544525 xlo xhi
-0.7112413733973663 4641.635525013436 ylo yhi
-0.31040563840239827 94.38056001840205 zlo zhi

Masses

1 69.723
2 15.999

Atoms # atomic

12044 1 16.48084810989415 55.42932471841946 38.12505780764583 0 0 0
1931310 2 48.18525291586019 27.831798493184593 80.92918814396762 0 0 0
974244 1 27.039751314453902 55.42660205698291 17.22602847071255 0 0 0
13006 1 29.03557367436082 55.61968375389696 11.449778029230568 0 0 0
1931770 2 48.31274734544035 30.53598071456587 21.985204562231615 0 0 0
12187 1 10.441409889206923 56.87254147028559 51.57747800529151 0 0 0
1927850 2 48.154888712937506 12.06369699409739 15.886144904961377 0 0 0

#RBS data file example:
#25200000
#Lattice="215.00000 0.00000 0.00000 0.00000 214.4 0.00000 0.00000 0.00000 5824.86343385"
#Ga -79.5354212843 -89.9262116686 -2899.4576111988  1
#Ga -80.8037735621 -88.2192198017 -2890.5819782791  1
#Ga -80.8037735621 -88.2192198017 -2890.5819782791  1

In [6]:
import re

def lammps_to_rbs(lammps_file, rbs_file):
    # Initialize section flags
    masses_section = False
    atoms_section = False

    # Parse the LAMMPS data file
    with open(lammps_file, 'r') as f:
        lines = f.readlines()
    
    # Variables to store cell dimensions and atoms data
    atoms = []
    lattice = []
    masses = {}
    
    # Process each line of the LAMMPS file
    for line in lines:
        # Stop reading if the line includes "velocity"
        if "velocities" in line.lower():
            break
        # Get box dimensions
        if "xlo xhi" in line:
            xlo, xhi = map(float, re.findall(r"[-+]?\d*\.\d+|\d+", line))
            lattice.append(xhi - xlo)
        elif "ylo yhi" in line:
            ylo, yhi = map(float, re.findall(r"[-+]?\d*\.\d+|\d+", line))
            lattice.append(yhi - ylo)
        elif "zlo zhi" in line:
            zlo, zhi = map(float, re.findall(r"[-+]?\d*\.\d+|\d+", line))
            lattice.append(zhi - zlo)
        
        # Get atomic masses (for identifying element names)
        elif "Masses" in line:
            masses_section = True
        elif masses_section and line.strip() and "Atoms" not in line:
            atom_type, mass = line.split()[:2]
            masses[int(atom_type)] = float(mass)
        elif "Atoms" in line:
            masses_section = False
            atoms_section = True
        elif atoms_section and line.strip():
            atom_data = line.split()
            atom_id = int(atom_data[0])
            atype = int(atom_data[1])
            atom_type =1 if atype<3 else 2
            x, y, z = map(float, atom_data[2:5])
            # Map type to element symbol (adjust based on actual element types in LAMMPS)
            element = "Ga" if atom_type == 1 else "O"
            atoms.append((element, x, y, z,atom_type))
    # Write to the RBS file
    with open(rbs_file, 'w') as f:
        # Write total number of atoms
        f.write(f"{len(atoms)}\n")
        # Write lattice dimensions, swapping y and z
        f.write(f'Lattice="{lattice[0]:.5f} 0.00000 0.00000 0.00000 {lattice[2]:.5f} 0.00000 0.00000 0.00000 {lattice[1]:.5f}"\n')
        
        # Write each atom in RBS format
        for atom in atoms:
            element, x, z, y, atom_type= atom  # Use swapped y and z
            f.write(f"{element} {x:.4f} {y:.4f} {z:.4f} {atom_type}\n")

# Example usage
for i in range(1,1102,100):
    lammps_file = '/Users/ruhe/Desktop/ga2o3/rbs/overlap/beta/cells/data.lastframe-{}'.format(i)   # Replace with your LAMMPS data file path
    rbs_file = '/Users/ruhe/Desktop/ga2o3/rbs/overlap/beta/cells/data{}.in'.format(i)        # Desired RBS output file path
    lammps_to_rbs(lammps_file, rbs_file)
